In [92]:
import pandas as pd
import numpy as np
X_train=np.load('x_train_mawilab.npy')
X_test=np.load('x_test_mawilab.npy')
y_train=np.load('y_train_mawilab.npy')#pd.read_csv('../data/maiwilab300000.tsv',sep='\t')['label']
y_test=np.load('y_test_mawilab.npy')
x=np.concatenate((X_train,X_test),axis=0)
y=np.concatenate((y_train,y_test),axis=0)
x=pd.DataFrame(x)
cols_to_norm = [ i for i in range(0,63) ]

x.loc[:, cols_to_norm] = (x[cols_to_norm] - x[cols_to_norm].mean()) / x[cols_to_norm].std()
x=x.values
X_train=x[:87500]
X_test=x[87500:]
y_test=y[87500:]

In [93]:

from sklearn.metrics import roc_curve, auc

import sklearn as sk

def evaluate(actual, predictions):
    FPR, TPR, thresholds = roc_curve(actual, predictions)
    cen = auc(FPR, TPR) 
    RightIndex=(TPR+(1-FPR)-1)
    index=np.argmax(RightIndex)
    tpr_val=TPR[index]
    fpr_val=FPR[index]
    thresholds_val=thresholds[index]
    y_pred=[0 if i<thresholds_val else 1 for i in predictions]
    y_pred_test=y_pred
    y_test_test=actual
    pre=sk.metrics.precision_score(y_test_test, y_pred_test)
    rec=sk.metrics.recall_score(y_test_test, y_pred_test)
    f1= sk.metrics.f1_score(y_test_test, y_pred_test)
    print("AUC", cen)
    print("Precision", pre)
    print( "Recall",rec)
    print( "f1_score", f1)
    mat=sk.metrics.confusion_matrix(y_test_test, y_pred_test)
    tp=mat[1][1]
    fn=mat[1][0]
    fp=mat[0][1]
    tn=mat[0][0]
    print("TPR",tp/(tp+fn))
    print("FPR",fp/(tn+fp))


In [94]:
n_classes=len(np.unique(y))

In [95]:
from sklearn.mixture import GaussianMixture
classifiers = dict((covar_type, GaussianMixture(n_components=n_classes,
                    covariance_type=covar_type,max_iter=200, random_state=0,init_params='kmeans',warm_start=True))
                   for covar_type in ['spherical', 'diag'])
n_classifiers = len(classifiers)

In [97]:
import collections

for index, (name, classifier) in enumerate(classifiers.items()):
    print(name)
    # Since we have class labels for the training data, we can
    # initialize the GMM parameters in a supervised manner.
    classifier.means_init = np.array([X_test[y_test == i].mean(axis=0) for i in range(n_classes)])
    
    # Train the other parameters using the EM algorithm.
    classifier.fit(X_test)
    y_test_pred = classifier.predict(X_test)
    y_test_pred2=[]
    for i in y_test_pred:
        if i==0:
            y_test_pred2.append(0)
        else:
            y_test_pred2.append(1)
            
    count=collections.Counter(y_test_pred2 )
    count2=collections.Counter(y_test)
    print(count,count2)
    evaluate(y_test,y_test_pred2)
#     test_accuracy = np.mean(y_test_pred.ravel() == y_test.ravel()) * 100
#     #evaluate(y_test,y_test_pred)
#     print(test_accuracy)


spherical
Counter({0: 8713, 1: 3787}) Counter({0.0: 7379, 1.0: 5121})
AUC 0.8573472236148654
Precision 0.9801954053340375
Recall 0.7248584260886546
f1_score 0.8334081724292771
TPR 0.7248584260886546
FPR 0.010163978858923974
diag
Counter({0: 12500}) Counter({0.0: 7379, 1.0: 5121})
AUC 0.5
Precision 0.0
Recall 0.0
f1_score 0.0
TPR 0.0
FPR 0.0


/root/anaconda3/envs/lr/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [168]:
np.mean(y_train_pred.ravel() == y_train.ravel())

0.5

In [99]:
np.mean(y_test_pred.ravel() == y_test.ravel())

0.97008

In [100]:
y_test_pred.ravel() == y_test.ravel()

array([ True,  True,  True, ..., False, False, False])

In [101]:
y_test_pred.ravel()

array([0, 0, 0, ..., 0, 0, 0])

In [3]:
X_train.shape

(800, 2)

In [4]:
Y_pred

array([3, 1, 0, 5, 3, 3, 5, 1, 1, 3, 0, 5, 0, 1, 1, 0, 1, 1, 0, 3, 0, 3,
       3, 0, 0, 5, 1, 1, 3, 1, 1, 1, 0, 3, 1, 1, 3, 1, 0, 0, 0, 1, 3, 1,
       1, 1, 0, 3, 3, 0, 1, 1, 1, 1, 1, 3, 1, 0, 0, 5, 0, 0, 1, 3, 3, 1,
       3, 1, 1, 3, 3, 1, 3, 0, 1, 3, 3, 3, 3, 0, 0, 0, 5, 0, 0, 0, 0, 3,
       0, 1, 3, 0, 0, 3, 3, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 3, 3, 0, 3, 0,
       1, 0, 1, 0, 3, 3, 1, 3, 0, 1, 0, 3, 0, 1, 0, 3, 3, 0, 0, 3, 0, 0,
       0, 1, 1, 3, 1, 1, 1, 3, 0, 3, 1, 1, 0, 3, 3, 3, 3, 3, 0, 3, 0, 0,
       3, 1, 1, 1, 1, 1, 1, 3, 3, 3, 3, 3, 0, 0, 0, 0, 3, 1, 1, 0, 1, 0,
       1, 3, 3, 3, 0, 3, 1, 1, 0, 3, 5, 3, 3, 1, 1, 3, 3, 5, 3, 3, 0, 0,
       3, 1])